In [9]:
import os
import numpy as np
from langchain.docstore.document import Document
from langchain_community.retrievers import BM25Retriever
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.retrievers import EnsembleRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
# Initialize the LLM
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

folder_path = "document"  # Replace with the path to your folder containing PDFs

In [2]:
def load_documents_from_txt_folder(folder_path):
    """
    Load all TXT files from a specified folder and return them as a list of LangChain Documents.
    Args:
        folder_path (str): Path to the folder containing the TXT files.
    Returns:
        List[Document]: List of LangChain Documents.
    """
    documents = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            # Read text content from the file
            with open(file_path, "r", encoding="utf-8") as file:
                text_content = file.read()
            # Create a LangChain Document for each file
            doc = Document(page_content=text_content,
                           metadata={"source": file_name})
            documents.append(doc)
    return documents

In [3]:
# --- Step 2: Implement Small2Big Chunking ---
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document


def small2big_chunking(documents, min_chunk_size=200, max_chunk_size=1000):
    """
    Apply the Small2Big chunking strategy to a list of documents and provide statistics.

    Args:
        documents (List[Document]): List of documents to chunk.
        min_chunk_size (int): Minimum chunk size.
        max_chunk_size (int): Maximum chunk size.

    Returns:
        List[Document]: List of chunked documents.
    """
    chunked_docs = []
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=min_chunk_size, chunk_overlap=50)

    total_original_length = 0
    total_chunked_length = 0

    for doc in documents:
        # Keep track of the original length of the document
        total_original_length += len(doc.page_content)

        # Split into small chunks first
        small_chunks = text_splitter.split_text(doc.page_content)
        combined_chunk = ""

        for chunk in small_chunks:
            if len(combined_chunk) + len(chunk) <= max_chunk_size:
                combined_chunk += " " + chunk
            else:
                chunked_docs.append(
                    Document(page_content=combined_chunk.strip(), metadata=doc.metadata))
                combined_chunk = chunk

        # Add the last chunk if it's not empty
        if combined_chunk:
            chunked_docs.append(
                Document(page_content=combined_chunk.strip(), metadata=doc.metadata))

    # Calculate statistics
    total_chunked_length = sum(len(doc.page_content) for doc in chunked_docs)
    avg_chunk_size = total_chunked_length / \
        len(chunked_docs) if chunked_docs else 0

    print("=== Small2Big Chunking Statistics ===")
    print(f"Total Original Documents: {len(documents)}")
    print(f"Total Original Length: {total_original_length} characters")
    print(f"Total Chunks Created: {len(chunked_docs)}")
    print(f"Total Chunked Length: {total_chunked_length} characters")
    print(f"Average Chunk Size: {avg_chunk_size:.2f} characters")
    print(f"Minimum Chunk Size: {min_chunk_size} characters")
    print(f"Maximum Chunk Size: {max_chunk_size} characters")

    return chunked_docs

In [4]:
# --- Step 3: Multi-Query Expansion ---
from langchain.output_parsers import PydanticToolsParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate


class ParaphrasedQuery(BaseModel):
    """You have performed query expansion to generate a paraphrasing of a question."""
    paraphrased_query: str = Field(
        ...,
        description="A unique paraphrasing of the original question."
    )


system = """
You are an expert in query paraphrasing and expansion. 

Your task is to generate multiple different phrasings of the same user query. 
Ensure that each paraphrased query captures the original meaning while using different wording.

If there are multiple common ways to phrase the question, or common synonyms for key terms, ensure all are included.

You **must** return at least 3 distinct paraphrased versions of the input query.
"""


prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "{question}"),
])

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_with_tools = llm.bind_tools([ParaphrasedQuery])
query_analyzer = prompt | llm_with_tools | PydanticToolsParser(tools=[
                                                               ParaphrasedQuery])


def expand_query(query):
    """
    Generate multiple semantically similar queries using an LLM.
    Args:
        query (str): Original query.
    Returns:
        List[str]: List of expanded queries.
    """
    try:
        response = query_analyzer.invoke({"question": query})
        # Extract paraphrased queries
        expanded_queries = [
            q.paraphrased_query for q in response if isinstance(q, ParaphrasedQuery)]

        if not expanded_queries or len(expanded_queries) < 3:
            print(
                "Warning: Less than 3 paraphrased queries returned. Consider adjusting the prompt.")

        print(f"Expanded Queries: {expanded_queries}\n")
        return expanded_queries

    except Exception as e:
        print(f"Error during query expansion: {e}")
        return []

d:\WorkSpace\venv\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# --- Step 4: Combine Results from Multi-Query Retrieval ---
def multi_query_retrieval(expanded_queries, retriever, top_k=5):
    """
    Perform retrieval using multiple queries and combine results.
    """
    combined_results = []
    for query in expanded_queries:
        results = retriever.get_relevant_documents(query)[:top_k]
        combined_results.extend(results)

    # Deduplicate results based on document content
    unique_docs = {doc.page_content: doc for doc in combined_results}
    return list(unique_docs.values())

In [6]:
from sklearn.metrics.pairwise import cosine_similarity



def rank_with_cove(query, retrieved_docs, embeddings, top_k=5):
    """




    Rank retrieved documents based on relevance using embeddings.





    Args:




        query (str): The user's input query.




        retrieved_docs (List[Document]): List of retrieved documents.




        embeddings: The embedding model used for similarity calculations.




        top_k (int): The number of top documents to return.





    Returns:




        List[Document]: The top-ranked documents.




    """

    # Step 1: Generate embedding for the query


    query_embedding = embeddings.embed_query(query)


    # Step 2: Compute embeddings for the retrieved documents

    doc_contents = [doc.page_content for doc in retrieved_docs]

    doc_embeddings = embeddings.embed_documents(doc_contents)


    # Step 3: Calculate cosine similarity between query and document embeddings


    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]


    # Step 4: Rank documents by similarity score


    ranked_indices = sorted(range(len(similarities)),
                            key=lambda i: similarities[i], reverse=True)


    # Step 5: Select top-k documents


    ranked_docs = [retrieved_docs[i] for i in ranked_indices[:top_k]]


    return ranked_docs

In [7]:
def optimized_rag_chain(query, retriever, llm, embeddings, top_k=5):
    """
    Perform retrieval-augmented generation with query optimization.
    Args:
        query (str): The user's input query.
        retriever: The retrieval mechanism.
        llm: The language model.
        embeddings: The embedding model for ranking documents.
        top_k (int): The number of top documents to retrieve.

    Returns:
        dict: A dictionary with the generated response and retrieved documents.
    """
    # Step 1: Expand the query using the LLM
    expanded_queries = expand_query(query)

    # Step 2: Retrieve documents using expanded queries
    retrieved_docs = multi_query_retrieval(expanded_queries, retriever, top_k)

    # Step 3: Rank documents using CoVe
    ranked_docs = rank_with_cove(query, retrieved_docs, embeddings, top_k)

    # Step 4: Combine the retrieved documents into a single context
    context = "\n\n".join([doc.page_content for doc in ranked_docs])

    # Step 5: Define the system prompt with the context
    system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know.\n\n"
        f"{context}\n\n"
        "Question: {query}"
    )

    # Step 6: Format the final prompt for the LLM
    final_prompt = system_prompt.format(query=query)
    # print(final_prompt)
    # Step 7: Get the answer from the LLM
    response = llm(final_prompt)  # Assuming llm takes input as a dictionary

    return {"query": query, "response": response, "retrieved_docs": ranked_docs}

In [10]:
documents = load_documents_from_txt_folder(folder_path)

In [11]:
# Apply Small2Big chunking
chunked_documents = small2big_chunking(documents)

=== Small2Big Chunking Statistics ===
Total Original Documents: 1
Total Original Length: 6005 characters
Total Chunks Created: 7
Total Chunked Length: 6517 characters
Average Chunk Size: 931.00 characters
Minimum Chunk Size: 200 characters
Maximum Chunk Size: 1000 characters


In [12]:
# Initialize retrievers
bm25_retriever = BM25Retriever.from_documents(chunked_documents)

In [13]:
vectorstore = InMemoryVectorStore.from_documents(
    documents=chunked_documents, embedding=OpenAIEmbeddings()
)
vector_retriever = vectorstore.as_retriever()

# Combine retrievers using EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever], weights=[0.5, 0.5]
)

In [14]:
# List of queries
queries = [
    "Tell me the main points I should care about privacy policies.",
]

# Initialize LLM and embeddings
llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

# Initialize a list to store results
results_list = []

# Loop through each query, perform RAG, and store the results
for query in queries:
    result = optimized_rag_chain(
        query=query,
        retriever=ensemble_retriever,
        llm=llm,
        embeddings=embeddings,
        top_k=5,
    )
    # Append the result to the list
    results_list.append(result)

Expanded Queries: ['What are the key aspects to consider in privacy policies?', 'What are the important elements of privacy policies that I should pay attention to?', 'What are the main factors to focus on in privacy policies?']



C:\Users\vanch\AppData\Local\Temp\ipykernel_27840\3567895178.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)[:top_k]
C:\Users\vanch\AppData\Local\Temp\ipykernel_27840\4076487573.py:40: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(final_prompt)  # Assuming llm takes input as a dictionary


In [15]:
# Display results
for result in results_list:
    print("=== Query ===")
    print(result["query"])
    print(result["response"].pretty_repr())
    print("\n")

=== Query ===
Tell me the main points I should care about privacy policies.
================================== Ai Message ==================================

When considering privacy policies, here are the main points you should focus on:

1. **Information Collection and Use**: Understand what types of personal information are collected (such as email, name, phone number, address, cookies, and usage data) and how this information is used.

2. **Cookies**: Be aware of how cookies are used to enhance website experience and how you can control their use through browser settings.

3. **Data Accuracy and Responsibilities**: Recognize the importance of providing accurate information and your responsibility to inform the entity of any changes to your personal data.

4. **Data Security**: Ensure that the entity uses encryption and other security measures to protect your data both during transmission and while stored.

5. **Data Retention and Disposal**: Know how long your data is retained and 

In [16]:
for result in results_list:
    # Loop through the retrieved documents for this query
    for i, doc in enumerate(result["retrieved_docs"], start=1):
        if isinstance(doc, Document):
            print(f"--- Document {i} ---")
            print(f"Metadata: {doc.metadata}")
            print(f"Content: {doc.page_content}\n")

--- Document 1 ---
Metadata: {'source': 'privacy_policy.txt'}
Content: COOKIES

We use cookies to enhance your experience on our website. You can control the use of cookies through your web browser settings.

THIRD-PARTY WEBSITES THIRD-PARTY WEBSITES

Our website may contain links to third-party websites. We are not responsible for the privacy practices or content of those websites.

CHANGES TO PRIVACY POLICY CHANGES TO PRIVACY POLICY

We may update this Privacy Policy from time to time. The updated Privacy Policy will be posted on our website.

CONTACT US CONTACT US

If you have any questions about this Privacy Policy, please contact us through the customer portal or by email at

PURPOSEFUL USE ONLY We commit to only use personal information for the purposes identified in the entity's privacy policy.
Presight.io 2022 All Rights Reserved
Ho Chi Minh City, Vietnam
Singapore
Seattle, WA, USA

--- Document 2 ---
Metadata: {'source': 'privacy_policy.txt'}
Content: By Role
By Team

PRIVACY 